In [1]:
from ipynb.fs.full.sim import Simulator
from ipynb.fs.full.traffic import Frame
from ipynb.fs.full.logger import Logger
from abc import ABC, abstractmethod

In [2]:
class Switch(ABC):
    def __init__(self, name: str, simulator: Simulator, logger: Logger):
        self.name = name
        self.sim = simulator
        self.logger = logger

        self.input_ports = {}
        self.output_ports = {}

    def add_input_port(self, port_id: str):
        self.input_ports[port_id] = None

    def add_output_port(self, port_id: str, link):
        self.output_ports[port_id] = link

    def receive(self, frame: Frame, ingress_port: str):
        self.logger.log(self.sim.time, self.name, "RECEIVE", frame, ingress_port)
        self.process(frame, ingress_port)

    @abstractmethod
    def process(self, frame: Frame, ingress_port: str):
        pass

    def forward(self, frame: Frame, egress_port: str):
        self.logger.log(self.sim.time, self.name, "FORWARD", frame, egress_port)
        link = self.output_ports[egress_port]
        link.transmit(frame)

In [3]:
class BasicSwitch(Switch):
    def __init__(self, name, simulator: Simulator, logger: Logger, routing_table: dict):
        super().__init__(name, simulator, logger)
        self.routing_table = routing_table  # stream_id → output_port

    def process(self, frame: Frame, ingress_port: str):
        out_port = self.routing_table.get(frame.stream_id)
        if out_port is None:
            self.logger.log(self.sim.time, self.name, "DROP", frame, None)
            return
        self.forward(frame, out_port)

In [4]:
class SinkSwitch(Switch):
    def process(self, frame, ingress_port):
        self.logger.log(self.sim.time, self.name, "SINK", frame, ingress_port)

In [5]:
class Link:
    def __init__(self, simulator: Simulator, delay: float, target_switch: Switch, target_port: str):
        self.sim = simulator
        self.delay = delay
        self.target_switch = target_switch
        self.target_port = target_port

    def transmit(self, frame: Frame):
        self.sim.schedule(
            self.delay,
            self.target_switch.receive,
            frame,
            self.target_port
        )

In [6]:
# 2 types: 1 with fixed intervals and 1 with random intervals like random(start, stop, step)
class TrafficSource:
    def __init__(self, simulator: Simulator, start_time=0.0):
        self.sim = simulator
        self.start_time = start_time
        self.frame_counter = 0

    def start_stream(self, stream_id, out_link, interval, size, count=None):
        def send_frame():
            self.frame_counter += 1
            frame = Frame(
                frame_id=self.frame_counter,
                stream_id=stream_id,
                size=size,
                creation_time=self.sim.time
            )
            out_link.transmit(frame)

            if count is None or self.frame_counter < count:
                self.sim.schedule(interval, send_frame)

        self.sim.schedule(self.start_time, send_frame)